In [6]:
from a import EncoderUnwrapper
import time
from IPython.display import clear_output
from STservo import ST3215
servo_bus = ST3215(port='COM16')
dev_id = [1, 2, 3, 4, 5, 6]

In [7]:
servo_bus.ping(4)

(0, [])

In [8]:
servo_bus.readPosi([3,4])

{3: {'posi': 3054, 'velo': 0, 'load': 0},
 4: {'posi': 1687, 'velo': 0, 'load': 0}}

In [ ]:
class Corrector:
    def __init__(self, servo_bus=None, dev_id=[1, 2, 3, 4, 5, 6], abs_max_posi=4095):
        self.total_step = abs_max_posi + 1
        for _id in dev_id:
            servo_bus.__set_posi_corr__(dev_id=_id, step=0, save=False)
        posi_raw = servo_bus.readPosi(dev_id=dev_id)
        encoder = {_id: EncoderUnwrapper(curr_posi=posi_raw[_id]["posi"], abs_max_posi=abs_max_posi)
                    for _id in dev_id}
        for _id in dev_id:
            encoder[_id].min_step = abs_max_posi
            encoder[_id].max_step = 0
            encoder[_id].posi_raw = 0
        self.encoder = encoder
        self.dev_id  = dev_id
        print('initialized')

    def update(self):
        posi_raw = servo_bus.readPosi(dev_id=self.dev_id)
        for _id in self.dev_id:
            _posi_raw = posi_raw[_id]["posi"]
            self.encoder[_id].posi_raw = _posi_raw
            #print(_id, _posi_raw)
            self.encoder[_id].update(_posi_raw)
            min_step = self.encoder[_id].min_step
            max_step = self.encoder[_id].max_step
            posi = self.encoder[_id].posi
            if posi < min_step:
                self.encoder[_id].min_step = posi
            elif posi > max_step:
                self.encoder[_id].max_step = posi

    def show(self):
        print(f' id |  min |  max |  len | curr |  raw')
        for _id in self.dev_id:
            min  = self.encoder[_id].min_step
            max  = self.encoder[_id].max_step
            curr = self.encoder[_id].posi
            raw  = self.encoder[_id].posi_raw
            len  = max - min
            print(f'{_id:3d} |{min: 5d} |{max: 5d} |{len: 5d}|{curr: 5d}|{raw: 5d}')

    def save(self):
        MARGIN = 100
        conf = {}
        print(f' id |  min |  max | corr ')
        for _id in self.dev_id:
            min  = self.encoder[_id].min_step 
            max  = self.encoder[_id].max_step
            if max < 4096 and min > 0:
                # 说明安装完毕的舵机无论如何转动，都不越过零点
                corr = 0
            else:
                if   min > 2047: 
                elif min >= 0: 
                elif min < -2047: 
                elif min < 0: 
                else:
                    print('save error')
                    raise
            print(f'{_id:3d} |{min: 5d} |{max: 5d}|{corr: 5d}')
            conf[_id] = {'min':min, 'max':max, 'corr':corr}
        return {'config': conf}

c = Corrector(servo_bus=servo_bus, dev_id=[3, 4])#) 
try:
    while True:
        time.sleep(0.3)
        clear_output(wait=True)
        c.update()
        c.show()
except KeyboardInterrupt:
    clear_output(wait=True)
    config = c.save()['config']

In [ ]:
servo_bus.ser.inWaiting()

In [ ]:
 id |  min |  max |  len | curr |  raw
  1 | 3505 | 6234 | 2729| 4840|  744
  2 |   -5 | 2365 | 2370|   -2| 4093
  3 | -879 | 1348 | 2227| 1343| 1343
  4 | 2054 | 4385 | 2331| 4077| 4077
  5 | 2548 | 3956 | 1408| 3395| 3394
  6 | 2363 | 3835 | 1472| 2381| 2381
config: {1: {'min': 3505, 'max': 6234, 'corr': 3405}, 2: {'min': -5, 'max': 2365, 'corr': -105}, 3: {'min': -879, 'max': 1348, 'corr': -979}, 
         4: {'min': 2054, 'max': 4385, 'corr': 1954}, 5: {'min': 2548, 'max': 3956, 'corr': 0}, 6: {'min': 2363, 'max': 3835, 'corr': 0}}

In [ ]:
servo_bus.__change_addr__(0xFE, 1)

In [ ]:

dev_id = [3, 4]
try:
    while True:
        time.sleep(0.3)
        clear_output(wait=True)
        for _id in dev_id:
            status, corr = servo_bus.__get_posi_corr__(dev_id=_id)
            print(f'id: {_id}, corr: {corr}, posi: {servo_bus.readPosi(dev_id=[_id], debug=True)[_id]["posi"]}')  
except KeyboardInterrupt:
    pass

In [ ]:
2800 - 2047 + 100

In [ ]:
servo_bus.__set_posi_corr__(dev_id=4, step=2568-2047+100)
time.sleep(1)
servo_bus.readPosi(dev_id=[4])

In [ ]:
c = Corrector(servo_bus=servo_bus, dev_id=[3, 4]) 

In [ ]:
servo_bus.readPosi(dev_id=[4])

In [ ]:

c = Corrector(servo_bus=servo_bus, dev_id=[1, 2, 3, 4, 5, 6])
try:
    while True:
        time.sleep(0.3)
        clear_output(wait=True)
        c.update()
        c.show()
except KeyboardInterrupt:
    clear_output(wait=True)
    c.save()

In [ ]:
 id |  min |  max 
  1 | 2727 | 5436
  2 | 3505 | 5896
  3 | 3942 | 6155
  4 | 2466 | 4814
  5 | 2454 | 6266
  6 | 3599 | 4883
config: {1: {'min': 2727, 'max': 5436}, 2: {'min': 3505, 'max': 5896}, 3: {'min': 3942, 'max': 6155}, 
         4: {'min': 2466, 'max': 4814}, 5: {'min': 2454, 'max': 6266}, 6: {'min': 3599, 'max': 4883}}

In [ ]:
servo_bus.readPosi(dev_id=[1])

In [ ]:
min_step = 1
max_step = 5400
MARGIN = 100

if min_step < 0:
    corr = min_step
    
elif max_step > 4095:
    corr = max_step - 4095 - MARGIN
    if corr < 0: raise

corr

In [ ]:
max_step - 4095 - MARGIN

In [ ]:
if 5890 > 2048:
    servo_bus.__set_posi_corr__(dev_id=2, step=5890 - 4000)

In [ ]:
 id |  min |  max |  len | curr
  1 |-1386 | 1341 | 2727| -398 
 id |  min |  max |  len | curr
  1 | 2712 | 5438 | 2726| 4048
  2 | -587 | 1805 | 2392| -551
  3 | -151 | 2059 | 2210| 2033
  4 | 2467 | 4814 | 2347| 4471
  5 |-1645 | 2182 | 3827|  327
  6 | -521 |  809 | 1330| -497


In [ ]:
-1386 - 2712 

In [ ]:
if 3504 > 2048:
    print(servo_bus.__get_posi_corr__(dev_id=2))

In [ ]:
print('posi', servo_bus.readPosi(dev_id=2, debug=True))

In [ ]:
posi = 0x0FFF
servo.move2Posi(dev_id=4, posi=posi, velo=1800, acc=100)
servo.move2Posi(dev_id=3, posi=posi, velo=1800, acc=100)
time.sleep(1)
print('posi', servo.readPosi(dev_id=[3, 4], debug=True))
time.sleep(6)
print('posi', servo.readPosi(dev_id=[3, 4], debug=True))
time.sleep(3)
servo.move2Posi(dev_id=[3, 4], posi=[0, 0], velo=1800, acc=100)

In [ ]:
servo.__set_torque_mode__(dev_id=3, mode='torque')
servo.__set_posi_corr__(dev_id=3, step=2000)
servo.move2Posi(dev_id=[3], posi=[1000], velo=1800, acc=100)

In [ ]:
servo.move2Posi(dev_id=[3, 4], posi=[2000, 3500], velo=1800, acc=100)

In [ ]:
status, posi_corr = servo.__get_posi_corr__(dev_id=4)
posi_corr

In [ ]:
servo.__set_torque_mode__(dev_id=3, mode='free')

In [ ]:
t = time.time()
while True:
    posi = servo.readPosi(dev_id=[3], debug=True)[3]['posi']
    posi += 2500
    servo.move2Posi(dev_id=[4], posi=[posi], velo=1800, acc=100)
    time.sleep(0.1)
    if time.time() - t > 1:
        t = time.time()
        print(posi-2500, posi)